In [1]:
import numpy as np
import pandas as pd 
import seaborn as sns
import tensorflow as tf
from tensorflow.python.ops import rnn, rnn_cell
from sklearn.metrics import roc_auc_score, classification_report
import matplotlib.pyplot as plt
import pickle
%matplotlib inline

## 1. Read files

In [2]:
# labels
labels = pd.read_csv("dataScienceTask/T_stage.csv")
labels['Stage_Progress'] = labels['Stage_Progress'].astype(np.int32)


# lab measurements 
SBP = pd.read_csv("dataScienceTask/T_SBP.csv")
DBP = pd.read_csv("dataScienceTask/T_DBP.csv")
ldl = pd.read_csv("dataScienceTask/T_ldl.csv")
creatinine = pd.read_csv("dataScienceTask/T_creatinine.csv")
HGB = pd.read_csv("dataScienceTask/T_HGB.csv")
glucose = pd.read_csv("dataScienceTask/T_glucose.csv")

SBP = SBP.rename(columns={'value':"SBP"})
DBP = DBP.rename(columns={'value':"DBP"})
ldl = ldl.rename(columns={'value':"ldl"})
creatinine = creatinine.rename(columns={'value':"creatinine"})
HGB = HGB.rename(columns={'value':"HGB"})
glucose = glucose.rename(columns={'value':"glucose"})


# medication history
drug_cat = {} # bookkeeping of heart dieases drug and dieabetes drug
drug_cat['diabetes'] = ["metformin","canagliflozin","dapagliflozin"]
drug_cat['heart'] = ["atorvastatin","losartan","Simvastatin","metoprolol","valsartan",\
                     "atenolol","rosuvastatin","pravastatin","carvedilol","lovastatin",\
                     "olmesartan","bisoprolol","propranolol","irbesartan","nebivolol",\
                     "telmisartan","labetalol","pitavastatin","simvastatin"]

meds = pd.read_csv("dataScienceTask/T_meds.csv")
meds['heart'] = (meds.drug.isin(drug_cat['heart']).astype(np.int32)) # dosage of heart disease medication
meds['duration'] = meds.end_day - meds.start_day 
meds = meds.rename(columns={'start_day':"time"})
meds = meds[['id','time','heart','daily_dosage','duration']]


# demographics 
from sklearn.preprocessing import LabelEncoder
demo = pd.read_csv("dataScienceTask/T_demo.csv")
demo['Female'] = (demo['gender'] == 'Female').astype(np.int32)
demo['race'] = LabelEncoder().fit(demo.race.values).transform(demo.race.values)
demo = demo[['id','Female','age','race']]


# merge medication history with lab measurements
df = meds.copy()
rights = [SBP,DBP,ldl,creatinine,HGB,glucose]
for right in rights:
    df = df.merge(right,on=['id','time'],how='outer')
df = df.sort_values(['id','time']).reset_index(drop=True)
# merge with labels
df = df.merge(labels,on='id',how='left')
# merge with demographics 
df = df.merge(demo,on='id',how='left')
## output
f_labs,f_meds,f_demo = ['creatinine','glucose','SBP','DBP','ldl','HGB'],['heart','daily_dosage','duration'],['Female','age','race'],
f_others = ['id','time','Stage_Progress']

df = df[f_others+f_labs+f_meds+f_demo]
df.to_csv("df.csv",sep=',', index=False, header=True)
df.describe()

,id,time,Stage_Progress,creatinine,glucose,SBP,DBP,ldl,HGB,heart,daily_dosage,duration,Female,age,race
count,6823.000000,6823.000000,6823.000000,1481.000000,1598.000000,1855.000000,1867.000000,1301.000000,2061.000000,2181.000000,2181.000000,2181.000000,6823.000000,6823.000000,6823.000000
mean,146.601055,412.961894,0.356002,1.327076,6.720413,134.324755,79.569855,87.247932,13.832693,0.774415,276.653141,79.741862,0.554741,70.626997,3.385314
std,87.964163,307.245821,0.478851,0.355466,1.617688,14.753248,10.176755,28.357415,1.642781,0.418063,478.411077,58.766049,0.497031,9.145888,1.221210
min,0.000000,-78.000000,0.000000,0.240000,2.890000,91.990000,44.950000,26.100000,8.820000,0.000000,2.000000,1.000000,0.000000,46.000000,0.000000
25%,69.000000,182.000000,0.000000,1.080000,5.630000,124.745000,72.860000,66.930000,12.680000,1.000000,20.000000,30.000000,0.000000,65.000000,3.000000
50%,144.000000,374.000000,0.000000,1.290000,6.380000,133.680000,79.090000,83.670000,13.930000,1.000000,50.000000,90.000000,1.000000,72.000000,4.000000
75%,225.000000,571.000000,1.000000,1.530000,7.450000,143.440000,86.290000,105.090000,14.980000,1.000000,320.000000,90.000000,1.000000,78.000000,4.000000
max,299.000000,1429.000000,1.000000,3.020000,16.610000,211.090000,112.930000,198.590000,19.000000,1.000000,2550.000000,540.000000,1.000000,86.000000,4.000000


## 2. Preprocessing

In [3]:
n_step = 10
n_aug = 1
dropout = 0.
noise_ratio = 0.
feature_names = f_labs + f_meds + f_demo

# ============ optional: relative time step
# def delshift_nan(trunc,feature_names,n_step):
#     h,w = trunc.shape
#     for f in feature_names:
#         nonnan = trunc[f].values[~np.isnan(trunc[f].values)]
#         trunc.loc[:,f] = np.hstack(([np.nan]*h, nonnan))[-h:]
#     return trunc.tail(n_step).values

# data = pd.DataFrame(columns=df.columns,data=np.nan*np.zeros((n_step*300,len(df.columns))))
# for i,g in df.groupby('id'):
#     value = delshift_nan(g,feature_names,n_step)
#     if value.shape[0] < n_step:
#         value = np.vstack((np.nan*np.zeros((n_step-value.shape[0],value.shape[1])),value))
#     data[i*n_step:(i+1)*n_step] = value   

# data[f_demo+f_others] = np.array([df[f_demo+f_others].groupby('id').head(1).values[i//n_step] for i in range(data.shape[0])])
# data['time'] = np.array(range(n_step)*300)
# data.to_csv("data_rlt{}.csv".format(str(n_step)),sep=',', index=False, header=True)
# ============ relative time step


# # ============ optional: absolute time step
# data = pd.DataFrame(columns=df.columns,data=np.nan*np.zeros((300*n_step,len(df.columns))))
# for i,g in df.groupby('id'):
#     value = g.tail(n_step).values
#     if value.shape[0] < n_step:
#         value = np.vstack((np.nan*np.zeros((n_step-value.shape[0],value.shape[1])),value))   
#     data[i*n_step:(i+1)*n_step] = value
    
# data[f_demo+f_others] = np.array([df[f_demo+f_others].groupby('id').head(1).values[i//n_step] for i in range(data.shape[0])])   
# data['time'] = np.array(range(n_step)*300)        
# data.to_csv("data_abs{}.csv".format(str(n_step)),sep=',', index=False, header=True)
# # # ============ absolute time step


# data fusion
name = "data_rlt10.csv" ## chosen preprocessing method: relative time step, n_step = 10
data = pd.read_csv(name)
data['id'] = data['id'].astype(np.int64)

data2 = pd.DataFrame(columns=['no_aug']+list(data.columns),data=np.nan*np.zeros((300*n_step*n_aug,1+len(data.columns))))
for i,g in data.groupby('id'):
    value = []
    for j in range(n_aug):
        nonnan = (~np.isnan(g[data.columns])).astype(np.int32)
        noise = np.random.rand(n_step,len(data.columns))*noise_ratio# add noise
        choices = np.random.choice([np.nan,1],n_step*len(data.columns),p=[dropout,1-dropout]).reshape(n_step,len(data.columns))# random dropout
        value.append((g[data.columns].values+noise)*choices*nonnan)
    aug = np.hstack([[au]*n_step for au in range(n_aug)]).reshape(-1,1)
    value = np.hstack((aug,np.vstack(value)))
    data2[i*(n_step*n_aug):(i+1)*(n_step*n_aug)] = value  
    
data2[f_demo+f_others] = np.array([data[f_demo+f_others].groupby('id').head(1).values[i//(n_step*n_aug)] for i in range(data2.shape[0])])
data2['time'] = np.array(range(n_step)*300*n_aug)
data2.to_csv("aug_"+name,sep=',', index=False, header=True)

# minmax scaling
minmax_features = f_labs + f_meds + f_demo
for f in minmax_features:
    mx, mn = data2[~np.isnan(data2[f])][f].max(),data2[~np.isnan(data2[f])][f].min()
    print(f,mx,mn)
    data2[f] = (data2[f] - mn)/(mx - mn) 
data2 = data2.fillna(0)
print(data2.describe())
data2.to_csv("final.csv",sep=',', index=False, header=True)

('creatinine', 3.02, 0.23999999999999999)
('glucose', 16.609999999999999, 2.8900000000000001)
('SBP', 211.09, 91.989999999999995)
('DBP', 112.93000000000001, 44.950000000000003)
('ldl', 198.59, 26.100000000000001)
('HGB', 19.0, 8.8200000000000003)
('heart', 1.0, 0.0)
('daily_dosage', 2550.0, 2.0)
('duration', 540.0, 1.0)
('Female', 1.0, 0.0)
('age', 86.0, 46.0)
('race', 4.0, 0.0)
       no_aug           id        time  Stage_Progress   creatinine  \
count  3000.0  3000.000000  3000.00000     3000.000000  3000.000000   
mean      0.0   149.500000     4.50000        0.333333     0.192640   
std       0.0    86.616497     2.87276        0.471483     0.215164   
min       0.0     0.000000     0.00000        0.000000     0.000000   
25%       0.0    74.750000     2.00000        0.000000     0.000000   
50%       0.0   149.500000     4.50000        0.000000     0.000000   
75%       0.0   224.250000     7.00000        1.000000     0.375000   
max       0.0   299.000000     9.00000        1.0

## 3. KNN

In [ ]:
## KNN input
# from sklearn.neighbors import KNeighborsClassifier
# data = pd.read_csv("final.csv")
# feature_names = f_demo

# features = np.array([g[feature_names].head(1).values.reshape(-1) for _,g in data.groupby('id')])
# labels = np.array([g['Stage_Progress'].head(1).values[0] for _,g in data.groupby('id')])
# train_test_split = np.load("train_test_split.npy") 
# train_x = features[train_test_split]
# train_y = labels[train_test_split]
# test_x = features[~train_test_split]
# test_y = labels[~train_test_split]

#============= optional: generate model complexity curve for KNN model
# dic = {"test_acc":[],"train_acc":[],"test_auc":[],"train_auc":[]} 
# limit=50
# for i in range(1,limit):
#     knn = KNeighborsClassifier(i)
#     pred_test = knn.fit(train_x,train_y).predict_proba(test_x)
#     pred_train = knn.fit(train_x,train_y).predict_proba(train_x)
    
#     train_acc = np.mean(train_y == np.argmax(pred_train,axis=1))
#     test_acc = np.mean(test_y == np.argmax(pred_test,axis=1))
#     train_auc = roc_auc_score(train_y,pred_train[:,1])
#     test_auc = roc_auc_score(test_y,pred_test[:,1])   
#     dic['train_acc'].append(train_acc)
#     dic['test_acc'].append(test_acc)
#     dic['train_auc'].append(train_auc)
#     dic['test_auc'].append(test_auc)   
    
# plt.style.use('default')
# plt.figure(figsize=(5,5))
# plt.plot(range(1,limit),dic['train_acc'],label='training')
# plt.plot(range(1,limit),dic['test_acc'],label='testing')
# plt.xlabel("Number of Neighbors")
# plt.ylabel("Accuracy")
# plt.legend()
# plt.show()

# plt.figure(figsize=(5,5))
# plt.plot(range(1,limit),dic['train_auc'],label='training')
# plt.plot(range(1,limit),dic['test_auc'],label='testing')
# plt.xlabel("Number of Neighbors")
# plt.ylabel("AUC score")
# plt.legend()
# plt.show()

#============= optional: generate predictions from selected KNN model
# knn = KNeighborsClassifier(7)
# pred_train = knn.fit(train_x,train_y).predict_proba(train_x)
# pred_test = knn.fit(train_x,train_y).predict_proba(test_x)
# np.save("knn_train.npy",pred_train[:,1])
# np.save("knn_test.npy",pred_test[:,1])

## 4. LSTM

In [5]:
data = pd.read_csv("final.csv")

feature_names =  f_meds # f_labs
n_input = len(feature_names)

learning_rate = 0.002
training_epochs = 1000
batch_size = 200
n_hidden = 64
n_classes = 2
alpha = 0.5

## split
features = np.array([g[feature_names].values for _,g in data.groupby('id')])
labels = np.array([g.head(1)['Stage_Progress'].values[0] for _,g in data.groupby('id')])
train_test_split = np.load("train_test_split.npy") #np.random.rand(300) < 0.80
train_x = features[train_test_split]
train_y = labels[train_test_split]
test_x = features[~train_test_split]
test_y = labels[~train_test_split]

train_x = np.vstack(train_x).reshape(-1,n_step,len(feature_names))
test_x = np.vstack(test_x).reshape(-1,n_step,len(feature_names))
train_y = pd.get_dummies(np.array([[i]*n_aug for i in train_y]).reshape(-1)).values
test_y = pd.get_dummies(np.array([[i]*n_aug for i in test_y]).reshape(-1)).values
total_batches = (train_x.shape[0]//batch_size)

print("n_step",n_step)
print("no of features",len(feature_names))
print("feature_names",feature_names)
print("train_x", train_x.shape)
print("train_y", train_y.shape)
print("test_x", test_x.shape)
print("test_y", test_y.shape)

tf.reset_default_graph()
# weights init
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev = 0.1)
    return tf.Variable(initial)
# bias init
def bias_variable(shape):
    initial = tf.constant(0.0, shape = shape)
    return tf.Variable(initial)
# model
def LSTM(x, weight, bias):
    multi_layer_cell = tf.nn.rnn_cell.MultiRNNCell([rnn_cell.LSTMCell(n_hidden, state_is_tuple=True) for _ in range(2)])
    output, state = tf.nn.dynamic_rnn(multi_layer_cell, x, dtype = tf.float32)
    output_flattened = tf.reshape(output, [-1, n_hidden])
    output_logits = tf.add(tf.matmul(output_flattened,weight),bias)
    output_all = tf.nn.sigmoid(output_logits)
    output_reshaped = tf.reshape(output_all,[-1,n_step,n_classes])
    output_last = tf.gather(tf.transpose(output_reshaped,[1,0,2]), n_step - 1)  
    return output_last, output_all
x = tf.placeholder("float", [None, n_step, n_input])
y = tf.placeholder("float", [None, n_classes])
y_steps = tf.placeholder("float", [None, n_classes])
weight = weight_variable([n_hidden,n_classes])
bias = bias_variable([n_classes])
y_last, y_all = LSTM(x,weight,bias)
# loss function
all_steps_cost = -tf.reduce_mean((y_steps * tf.log(y_all))  + (1 - y_steps) * tf.log(1 - y_all))
last_step_cost = -tf.reduce_mean((y * tf.log(y_last)) + ((1 - y) * tf.log(1 - y_last)))
loss_function = (alpha * all_steps_cost) + ((1 - alpha) * last_step_cost)
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(loss_function)

('n_step', 10)
('no of features', 3)
('feature_names', ['heart', 'daily_dosage', 'duration'])
('train_x', (239, 10, 3))
('train_y', (239, 2))
('test_x', (61, 10, 3))
('test_y', (61, 2))


/Users/luwang/anaconda/lib/python2.7/site-packages/tensorflow/python/ops/gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


## 4. Training and testing

In [6]:
# model selection
threshold = 0.65
for round in range(10):
    print("round",round)
    save_dic = False
    with tf.Session() as session:
        tf.global_variables_initializer().run()
        dic = {"test_acc":[],"train_acc":[],"test_auc":[],"train_auc":[]} #record
        for epoch in range(training_epochs):
            for b in range(total_batches): 
                choices = np.random.choice(range(len(train_x)),batch_size,replace=True)#random sampling with replacement
                batch_x, batch_y = train_x[choices], train_y[choices]
                batch_y_steps = np.tile(batch_y,((train_x.shape[1]),1))
                _, c = session.run([optimizer, loss_function],feed_dict={x: batch_x, y : batch_y, y_steps: batch_y_steps})   

            pred_train_y = session.run(y_last,feed_dict={x:train_x})
            pred_test_y = session.run(y_last,feed_dict={x:test_x})
            train_acc = np.mean(train_y[:,1] == np.argmax(pred_train_y,axis=1))
            test_acc = np.mean(test_y[:,1] == np.argmax(pred_test_y,axis=1))
            train_auc = roc_auc_score(train_y,pred_train_y)
            test_auc = roc_auc_score(test_y,pred_test_y)
            dic['train_acc'].append(train_acc)
            dic['test_acc'].append(test_acc)
            dic['train_auc'].append(train_auc)
            dic['test_auc'].append(test_auc)
            
            if (test_acc > threshold) and (test_auc > threshold):
                report = classification_report(test_y[:,1],np.argmax(pred_test_y,axis=1),target_names=['Stable','Progress'])
                save_name = 'round{}_model{}_{}_{}'.format(str(round),str(epoch),str(test_acc),str(test_auc))
                print("epoch",epoch)
                print(report)
                print("Accuracy :",test_acc,train_acc)
                print("AUC Score:",test_auc,train_auc)
                saver = tf.train.Saver()
                saver.save(session, 'mymodel/'+save_name+".ckpt")
                with open('mymodel/'+save_name+"report.pkl".format(str(round),str(epoch),str(test_acc),str(test_auc)), 'wb') as handle:
                    pickle.dump(report, handle, protocol=pickle.HIGHEST_PROTOCOL)
                save_dic = True
                threshold+=0.01
        if save_dic:
            with open('mymodel/'+save_name+"dic.pkl".format(str(round),str(epoch),str(test_acc),str(test_auc)), 'wb') as handle:
                pickle.dump(dic, handle, protocol=pickle.HIGHEST_PROTOCOL)

('round', 0)
('epoch', 498)
             precision    recall  f1-score   support

     Stable       0.70      0.88      0.78        40
   Progress       0.55      0.29      0.37        21

avg / total       0.65      0.67      0.64        61

('Accuracy :', 0.67213114754098358, 0.78242677824267781)
('AUC Score:', 0.6511904761904761, 0.77958860759493676)
('round', 1)
('round', 2)
('round', 3)
('epoch', 296)
             precision    recall  f1-score   support

     Stable       0.68      1.00      0.81        40
   Progress       1.00      0.10      0.17        21

avg / total       0.79      0.69      0.59        61

('Accuracy :', 0.68852459016393441, 0.67782426778242677)
('AUC Score:', 0.66309523809523807, 0.64109968354430391)
('epoch', 329)
             precision    recall  f1-score   support

     Stable       0.67      0.97      0.80        40
   Progress       0.67      0.10      0.17        21

avg / total       0.67      0.67      0.58        61

('Accuracy :', 0.67213114754098

In [ ]:
#============= optional: generate model complexity curve for selected LSTM model

## with open('mymodel/f_labs_rlt10/round242_model631_0.852459016393_0.836904761905report.pkl', 'rb') as handle:#LSTM11
## with open("mymodel/f_labs_rlt10/round58_model488_0.819672131148_0.832738095238report.pkl",'rb') as handle:#LSTM12
# with open('mymodel/f_meds_rlt10/2layer/round87_model708_0.737704918033_0.71369047619report.pkl', 'rb') as handle:#LSTM2
## with open('mymodel/f_labs_f_meds_rlt10/round46_model234_0.770491803279_0.75119047619report.pkl', 'rb') as handle:#LSTM
#     report = pickle.load(handle)
#     print(report)
## with open('mymodel/f_labs_rlt10/round242_model631_0.852459016393_0.836904761905dic.pkl', 'rb') as handle:#LSTM11
## with open("mymodel/f_labs_rlt10/round58_model488_0.819672131148_0.832738095238dic.pkl",'rb') as handle:#LSTM12
# with open('mymodel/f_meds_rlt10/2layer/round87_model708_0.737704918033_0.71369047619dic.pkl', 'rb') as handle:#LSTM2
## with open('mymodel/f_labs_f_meds_rlt10/round46_model234_0.770491803279_0.75119047619dic.pkl', 'rb') as handle:#LSTM  
#     dic=pickle.load(handle)

# training_epochs=1000
# plt.style.use('default')
# plt.figure(figsize=(5,5))
# plt.plot(range(training_epochs),dic['train_acc'],label='training')
# plt.plot(range(training_epochs),dic['test_acc'],label='testing')
# plt.xlabel("Epoches")
# plt.ylabel("Accuracy")
# plt.legend()
# plt.show()

# plt.figure(figsize=(5,5))
# plt.plot(range(training_epochs),dic['train_auc'],label='training')
# plt.plot(range(training_epochs),dic['test_auc'],label='testing')
# plt.xlabel("Epoches")
# plt.ylabel("AUC score")
# plt.legend()
# plt.show()

#============= optional: generate predictions from selected LSTM model
# tf.reset_default_graph()
# x = tf.placeholder("float", [None, n_step, n_input])
# y = tf.placeholder("float", [None, n_classes])
# y_steps = tf.placeholder("float", [None, n_classes])
# weight = weight_variable([n_hidden,n_classes])
# bias = bias_variable([n_classes])
# y_last, y_all = LSTM(x,weight,bias)

# with tf.Session() as session:
#     saver = tf.train.Saver()
# #     saver.restore(session, "mymodel/f_labs_rlt10/round242_model631_0.852459016393_0.836904761905.ckpt")#LSTM11
# #     saver.restore(session,"mymodel/f_labs_rlt10/round58_model488_0.819672131148_0.832738095238.ckpt")#LSTM12
#    saver.restore(session, "mymodel/f_meds_rlt10/round47_model651_0.704918032787_0.697619047619.ckpt")#LSTM2
# #     saver.restore(session, "mymodel/f_labs_f_meds_rlt10/round46_model234_0.770491803279_0.75119047619.ckpt")#LSTM 
#     pred_train = session.run(y_last,feed_dict={x:train_x})
#     pred_test = session.run(y_last,feed_dict={x:test_x})
    
#     train_acc = np.mean(train_y[:,1] == np.argmax(pred_train,axis=1))
#     test_acc = np.mean(test_y[:,1] == np.argmax(pred_test,axis=1))
#     train_auc = roc_auc_score(train_y,pred_train)
#     test_auc = roc_auc_score(test_y,pred_test)
#     np.save("LSTM2_train.npy",pred_train[:,1])
#     np.save("LSTM2_test.npy",pred_test[:,1])

## 5. Final Model: LR

In [7]:
from sklearn.linear_model import LogisticRegression

knn_test = np.load("knn_test.npy")
knn_train = np.load("knn_train.npy")
LSTM11_test = np.load("LSTM11_test.npy")
LSTM11_train = np.load("LSTM11_train.npy")
LSTM12_test = np.load("LSTM12_test.npy")
LSTM12_train = np.load("LSTM12_train.npy")
LSTM2_test = np.load("LSTM2_test.npy")
LSTM2_train = np.load("LSTM2_train.npy")
LSTM_test = np.load("LSTM_test.npy")
LSTM_train = np.load("LSTM_train.npy")

train_x = np.vstack((knn_train,LSTM11_train,LSTM12_train,LSTM_train)).T
test_x = np.vstack((knn_test,LSTM11_test,LSTM12_test,LSTM_test)).T

LR = LogisticRegression()
pred_test = LR.fit(train_x,train_y[:,1]).predict_proba(test_x)
pred_train = LR.fit(train_x,train_y[:,1]).predict_proba(train_x)
print("Coefficients:",LR.coef_)

train_acc = np.mean(train_y[:,1] == np.argmax(pred_train,axis=1))
test_acc = np.mean(test_y[:,1] == np.argmax(pred_test,axis=1))
train_auc = roc_auc_score(train_y,pred_train)
test_auc = roc_auc_score(test_y,pred_test)   
report = classification_report(test_y[:,1],np.argmax(pred_test,axis=1),target_names=['Stable','Progress'])
print(report)
print("Accuracy :",test_acc)
print("AUC score:",test_auc)

('Coefficients:', array([[ 0.42025567,  4.66844874,  2.24842019, -0.23361715]]))
             precision    recall  f1-score   support

     Stable       0.92      0.82      0.87        40
   Progress       0.72      0.86      0.78        21

avg / total       0.85      0.84      0.84        61

('Accuracy :', 0.83606557377049184)
('AUC score:', 0.86071428571428577)
